## 텍스트와 시퀀스를 처리하는 기본적인 모델
- Recurrent Neural network
- 1D convnet

## 애플리케이션
- 문서, 시계열 분류 (글의 주제 혹은 저자 식별)
- 시계열 비교, 두 시퀀스가 얼마나 밀접한 관련이 있는지
- seq-to-seq 번역
- 감성 분석 (긍/부정)
- 시계열 예측, 어떤 지여그이 최근 날씨 데이터 -> 향후 날씨 예측
- 질의 응답

## 텍스트 데이터 다루기
- 단어의 시퀀스 혹은 문자의 시퀀스
- 문자 언어에 대한 통계적 구조를 만들어 간단한 문제 해결
- 자연어 처리를 위한 딥러닝은 단어, 문장, 문단에 대한 **패턴 인식**
- 텍스트를 처리하기 위해 **텍스트 벡터화(vectorizing text)** 해야함
    - 텍스트를 단어로 나누고 각 단어를 하나의 벡터로 변환
    - 텍스트를 문자로 나누고 각 문자를 하나의 벡터로 변환
    - 텍스트에서 단어나 문자의 n-gram을 추출하여 각 n-gram을 하나의 벡터로 변환
    

## 텍스트 벡터화
1. 토큰화(tokenization) 
2. 토큰에 수치형 벡터 연결
    - one-hot encoding
    - token embedding(word embedding; 일반적으로 단어에 적용하기 때문)

## n-gram, BoW
- word n-gram
    - { The, The cat, cat, cat sat, sat, sat on, on, on the, ... } bag of 2-gram
- Bag-of-Words ; 특정한 순서 x, 집합
- But, 얕은 학습법에 적합한 방법임, 순서 정보가 다 사라져버림

## 이후의 과정은 
- [원 핫 인코딩](./deep-learning-with-python/6.1-one-hot-encoding-of-words-or-characters.ipynb)
- [워드 임베딩](./deep-learning-with-python/6.1-using-word-embeddings.ipynb)

## RNN 이해하기

- 지금까지 살펴본 뉴럴넷에선 메모리가 없음
- 입력은 개별적으로 처리되고, 입력 간에 유지되는 상태란 없음
    - 이런 네트워크는 feedforward network라고 함
- 이와 달리, 이전에 나온 것을 기억하면서 처리할 수 가 있음
    - 이를 Recurrent Neural Network로 표현할 수 있음, 순환신경망
    - 시퀀스의 원소를 순회하면서 지금까지 처리한 정보를 상태(state)에 저장
        - 이 상태는 서로 다른 시퀀스를 처리 할 때는 재설정 됨
        - 하나의 시퀀스가 여전히 하나의 데이터 포인트로 간주됨
        - 대신 데이터 포인트가 한 번에 처리되는게 아니라 원소를 차례대로 처리함
    - RNN은 (timesteps, input_features) 모양의 2D 텐서로 인코딩된 벡터의 시퀀스를 입력받음
    - 이 시퀀스는 타임스텝을 따라 반복되며,
    - 각 타임스텝 t에서 **이전 상태와 현재 입력을 둘 다 고려해서 출력을 계산**
    - 그 다음, 이 출력을 다음 스텝의 상태로 설정
    - 반복

## RNN 정방향 계산 예시
- 이전 타임스텝의 출력을 은닉 상태라 부름(**Hidden State**)

In [4]:
import numpy as np

timesteps = 100
input_features = 32
output_features = 64

inputs = np.random.random((timesteps, input_features))

state_t = np.zeros((output_features,))

# weights matrix
W = np.random.random((output_features, input_features))
U = np.random.random((output_features, output_features))
b = np.random.random((output_features,))

successive_outputs = []
for input_t in inputs:    # input_t'shape == (input_features, )
    output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b)
    successive_outputs.append(output_t)
    state_t = output_t

# 최종 출력의 모양은 (timesteps, output_features)
final_output_sequence = np.stack(successive_outputs, axis=0)
final_output_sequence[-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

## 이후의 과정은
- [RNN](./deep-learning-with-python/6.2-understanding-recurrent-neural-networks.ipynb)

## LSTM, GRU 이해
- SimpleRNN은 이론적으로 시간 t에서 이전의 모든 타임스텝의 정보를 유지할 수 있지만
    - 실제로 긴 시간에 걸친 의존성은 학습할 수 없다
    - Vanishing Gradient Problem
        - RNN의 학습은 timesteps에 따라 네트워크를 펼쳐놓은 것 처럼 진행
        - BackPropagation Through Time
        - 구조가 재귀적이기 때문에, 타임스텝이 길어질 수록 그래디언트 값이 vanishing 혹은 exploding함
- Long Short Term Memory
    - RNN의 변종으로, 정보를 여러 타임스텝에 걸쳐 나르는 방법이 추가된다.
    - 처리되는 시퀀스와 나란히 컨베이어 벨트가 있다고 생각 (**Cell State**)
        - 시퀀스를 처리하다가 추출된 중요한 정보가 컨베이어로 올라가 필요한 시점에 끄집어 내 씀
        - **forget gate**가 정보를 적절히 망각을하고
        - **cell state**에 추가할 정보를 생성 및 input gate를 통해 불순물을 제거하고
        - forget gate에서 설정된 비율을 기존의 cell state에 곱하고 불순물이 제거된 새로운 정보가 합해져 **cell state가 업데이트**되고
        - **output gate**에 의해 설정된 비율을 통해 cell state의 정보를 필터링해서 아웃풋을 생성
            - 라고 하지만, 실제로 하는일은 셀의 가중치에 의해 결정됨, 가설일 뿐
    - 중요한건, 중요한 과거 정보를 유지하는 기능이 있다는 것

## 이후의 과정은
- [LSTM#케라스를 사용한 LSTM 예제부분](./deep-learning-with-python/6.2-understanding-recurrent-neural-networks.ipynb)